In [97]:
### Creating Combat Log Table

In [98]:
# what needs to happen is that a table needs to be created that stores each turn in combat and the actions that happened in that turn.
# the table should have the following columns:

# create the table in the form of a dataframe
import pandas as pd
import numpy as np
import sqlite3
    
# create the table
combat_log = pd.DataFrame(columns=[
                                   'Combat Round',
                                   'Action Number',
                                   'Action Time', # action, bonus action, reaction, etc
                                   'Action Name', # attack, cast spell, dodge, etc
                                   'Action Type', # attack, support, heal, control, etc
                                   'Target',
                                   'Action Result',
                                   'Current Allied Ability Check',
                                   'Current Allied Attack Roll',
                                   'Current Allied Saving Throw',
                                   'Current Allied Damage Roll',
                                   'Current Enemy Ability Check',
                                   'Current Enemy Attack Roll',
                                   'Current Enemy Saving Throw',
                                   'Current Enemy Damage Roll',
                                   ])

In [99]:
import Species
import Backgrounds
import Armor_and_Weapons
import Dice_Rolls
import Establishing_Hierarchy
import CHARACTER_CREATOR
import Monsters

import Character_Actions

In [100]:
def generate_situation(character_num):
    print('generating situation...')

    # for the number of characters, randomly generate a Player_Character using Random_Character()
    entities = []
    print('generating characters...')
    for i in range(character_num):
        entities.append(CHARACTER_CREATOR.Random_Character())


    # for the number of entities, within each entity's flesh out the Active_Conditions and Circumstances dictionaries
    for entity in range(len(entities)):
        other_entities = entities.copy()
        other_entities.pop(entity)
        
        # I want to add a key for each other entity in the Active_Conditions dictionaries
        for other_entity in range(len(other_entities)):
            entities[entity].Active_Conditions[str(entities[other_entity].Name)] = []
        
    return entities



def generate_combat(character_num):
    print('generating combat...')

    combat_log_new = pd.DataFrame(columns=[
                                   'Combat Round',
                                   'Action Number',
                                   'Action Time', # action, bonus action, reaction, etc
                                   'Action Name', # attack, cast spell, dodge, etc
                                   'Action Type', # attack, support, heal, control, etc
                                   'Target',
                                   'Action Result',
                                   'Current Allied Ability Check',
                                   'Current Allied Attack Roll',
                                   'Current Allied Saving Throw',
                                   'Current Allied Damage Roll',
                                   'Current Enemy Ability Check',
                                   'Current Enemy Attack Roll',
                                   'Current Enemy Saving Throw',
                                   'Current Enemy Damage Roll',
                                   ])
    # create situation
    combat_situation = generate_situation(character_num)
    

    print('adding action options...')
    for entity in range(len(combat_situation)):
        combat_situation[entity].Actions['None'] = Character_Actions.No_Action(combat_situation[entity],combat_situation,combat_log_new)
        combat_situation[entity].Bonus_Actions['None'] = Character_Actions.No_Bonus_Action(combat_situation[entity],combat_situation,combat_log_new)
        combat_situation[entity].Actions['Attack'] = Character_Actions.Attack_Action(combat_situation[entity],combat_situation,combat_log_new)
        combat_situation[entity].Actions['Dodge'] = Character_Actions.Dodge_Action(combat_situation[entity],combat_situation,combat_log_new)

        Character_Actions.Equip_Weapon(combat_situation[entity],Character_Actions.Choose_Random_Weapon(combat_situation[entity]))

    combat_round = 0
    combat_log_new['Combat Round'] = combat_round
    

    def take_turn(combat_log_new,character):
        # need to decide which action_type to take first: Actions or Bonus_Actions
        action_types = ['Actions','Bonus_Actions']
        first_action_type = np.random.choice(action_types)
        second_action_type = action_types[1-action_types.index(first_action_type)]

        if first_action_type == 'Actions':
            first_action = np.random.choice(['None','Attack','Dodge'])
            new_turn = combat_situation[character].Actions[first_action]

        elif first_action_type == 'Bonus_Actions':
            first_action = np.random.choice(['None'])
            new_turn = combat_situation[character].Bonus_Actions[first_action]
        else: pass
                
        if second_action_type == 'Actions':
            second_action = np.random.choice(['None','Attack','Dodge'])
            new_turn = pd.concat([new_turn, combat_situation[character].Actions[second_action]], ignore_index=True, axis=0)

        elif second_action_type == 'Bonus_Actions':
            second_action = np.random.choice(['None'])
            new_turn = pd.concat([new_turn, combat_situation[character].Bonus_Actions[second_action]], ignore_index=True, axis=0)
        else: pass
        
        #new_round = combat_situation[character].first_action_type[first_action]
        #new_round = pd.concat([new_round, combat_situation[character].second_action_type[second_action]], ignore_index=True, axis=0)


        #action = np.random.choice(['None','Attack','Dodge'])
        #bonus_action = np.random.choice(['None'])
        #new_round = combat_situation[character].Actions[action]
        #combat_log_new = pd.concat([combat_log_new, new_round], ignore_index=True, axis=0)
        return new_turn

    print('taking turns...')
    for entity in range(len(combat_situation)):
        combat_log_new = pd.concat([combat_log_new,take_turn(combat_log_new,entity)], ignore_index=True, axis=0)


    combat_log_new = combat_log_new.reindex(columns=['Combat Round'] + list(combat_log_new.columns[1:][combat_log_new.columns[1:].str.contains('Acting True')]) + list(combat_log_new.columns[1:][combat_log_new.columns[1:].str.contains('Current_HP')]) + list(combat_log_new.columns[1:][~combat_log_new.columns[1:].str.contains('Acting True') & ~combat_log_new.columns[1:].str.contains('Current_HP')]))

    return combat_log_new


In [101]:
# remove the number of columns limit
pd.set_option('display.max_columns', None)

In [102]:
generate_combat(4)

generating combat...
generating situation...
generating characters...
Character_Creator - Random Character Function - Class Choice:  <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Artificer.py'>
Create Character Function - Class: <module 'Artificer' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Artificer.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Barbarian' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Barbarian.py'>
Create Character Function - Class: <module 'Barbarian' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Barbarian.py'>
Character_Creator - Random Character Function - Class Choice:  <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Create Character Function - Class: <module 'Bard' from 'c:\\Users\\maxhi\\OneDrive\\Documents\\GitHub\\5e-Simulation\\Bard.py'>
Character_Creator - Random Character F

,Combat Round,a9692 Acting True,a6579 Acting True,a5558 Acting True,a3644 Acting True,a9692 Current_HP,a6579 Current_HP,a5558 Current_HP,a3644 Current_HP,Action Number,Action Time,Action Name,Action Type,Target,Action Result,Current Allied Ability Check,Current Allied Attack Roll,Current Allied Saving Throw,Current Allied Damage Roll,Current Enemy Ability Check,Current Enemy Attack Roll,Current Enemy Saving Throw,Current Enemy Damage Roll,a9692 Temp_HP,a9692 Size,a9692 Walking Speed,a9692 Flying Speed,a9692 Str_Score,a9692 Dex_Score,a9692 Con_Score,a9692 Int_Score,a9692 Wis_Score,a9692 Cha_Score,a9692 Active_Conditions,a9692 Concentrating,a6579 Temp_HP,a6579 Size,a6579 Walking Speed,a6579 Flying Speed,a6579 Str_Score,a6579 Dex_Score,a6579 Con_Score,a6579 Int_Score,a6579 Wis_Score,a6579 Cha_Score,a6579 Active_Conditions,a6579 Concentrating,a5558 Temp_HP,a5558 Size,a5558 Walking Speed,a5558 Flying Speed,a5558 Str_Score,a5558 Dex_Score,a5558 Con_Score,a5558 Int_Score,a5558 Wis_Score,a5558 Cha_Score,a5558 Active_Conditions,a5558 Concentrating,a3644 Temp_HP,a3644 Size,a3644 Walking Speed,a3644 Flying Speed,a3644 Str_Score,a3644 Dex_Score,a3644 Con_Score,a3644 Int_Score,a3644 Wis_Score,a3644 Cha_Score,a3644 Active_Conditions,a3644 Concentrating
0,0,1.0,0.0,0.0,0.0,10.0,12.0,10.0,11.0,Undetermined,Bonus Action,None,None,a9692,None,<Dice_Rolls.Ability_Check object at 0x00000215...,<Dice_Rolls.Attack_Roll object at 0x0000021588...,<Dice_Rolls.Saving_Throw object at 0x000002158...,<Dice_Rolls.Damage_Roll object at 0x0000021588...,<Dice_Rolls.Ability_Check object at 0x00000215...,<Dice_Rolls.Attack_Roll object at 0x0000021588...,<Dice_Rolls.Saving_Throw object at 0x000002158...,<Dice_Rolls.Damage_Roll object at 0x0000021588...,0.0,Medium,30.0,0.0,12.0,13.0,10.0,8.0,15.0,14.0,NaN,False,0.0,Medium,30.0,0.0,13.0,8.0,15.0,10.0,14.0,12.0,NaN,False,0.0,Medium,30.0,0.0,14.0,15.0,12.0,12.0,8.0,13.0,NaN,False,0.0,Medium,30.0,0.0,14.0,11.0,13.0,9.0,16.0,15.0,NaN,False
1,0,1.0,0.0,0.0,0.0,10.0,12.0,7.0,11.0,Undetermined,Action,Attack,Offense,a5558,Hit,<Dice_Rolls.Ability_Check object at 0x00000215...,<Dice_Rolls.Attack_Roll object at 0x0000021588...,<Dice_Rolls.Saving_Throw object at 0x000002158...,<Dice_Rolls.Damage_Roll object at 0x0000021588...,<Dice_Rolls.Ability_Check object at 0x00000215...,<Dice_Rolls.Attack_Roll object at 0x0000021588...,<Dice_Rolls.Saving_Throw object at 0x000002158...,<Dice_Rolls.Damage_Roll object at 0x0000021588...,0.0,Medium,30.0,0.0,12.0,13.0,10.0,8.0,15.0,14.0,[],False,0.0,Medium,30.0,0.0,13.0,8.0,15.0,10.0,14.0,12.0,[],False,0.0,Medium,30.0,0.0,14.0,15.0,12.0,12.0,8.0,13.0,[],False,0.0,Medium,30.0,0.0,14.0,11.0,13.0,9.0,16.0,15.0,[],False
2,0,0.0,1.0,0.0,0.0,10.0,12.0,7.0,11.0,Undetermined,Action,Attack,Offense,a5558,Miss,<Dice_Rolls.Ability_Check object at 0x00000215...,<Dice_Rolls.Attack_Roll object at 0x0000021588...,<Dice_Rolls.Saving_Throw object at 0x000002158...,<Dice_Rolls.Damage_Roll object at 0x0000021588...,<Dice_Rolls.Ability_Check object at 0x00000215...,<Dice_Rolls.Attack_Roll object at 0x0000021588...,<Dice_Rolls.Saving_Throw object at 0x000002158...,<Dice_Rolls.Damage_Roll object at 0x0000021588...,0.0,Medium,30.0,0.0,12.0,13.0,10.0,8.0,15.0,14.0,[],False,0.0,Medium,30.0,0.0,13.0,8.0,15.0,10.0,14.0,12.0,[],False,0.0,Medium,30.0,0.0,14.0,15.0,12.0,12.0,8.0,13.0,[],False,0.0,Medium,30.0,0.0,14.0,11.0,13.0,9.0,16.0,15.0,[],False
3,0,0.0,1.0,0.0,0.0,10.0,12.0,7.0,11.0,Undetermined,Bonus Action,None,None,a6579,None,<Dice_Rolls.Ability_Check object at 0x00000215...,<Dice_Rolls.Attack_Roll object at 0x0000021588...,<Dice_Rolls.Saving_Throw object at 0x000002158...,<Dice_Rolls.Damage_Roll object at 0x0000021588...,<Dice_Rolls.Ability_Check object at 0x00000215...,<Dice_Rolls.Attack_Roll object at 0x0000021588...,<Dice_Rolls.Saving_Throw object at 0x000002158...,<Dice_Rolls.Damage_Roll object at 0x0000021588...,0.0,Medium,30.0,0.0,12.0,13.0,10.0,8.0,15.0,14.0,NaN,False,0.0,Medium,30.0,0.0,13.0,8.0,15.0,10.0,1